<a href="https://colab.research.google.com/github/tarvt/DeleSmell_SA_G5/blob/main/dataset_class/cassandra-trunk/cassandra_semantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import numpy as np
import re
from string import punctuation
import pandas as pd
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
save_file_csv = 'cassandra2.csv'
file_csv =  'cassandra.csv'
file_dir = '/content/drive/MyDrive/delesmell/dataset_class/cassandra-trunk' #Newly stored class address


In [4]:
fileList = [] # list of all the cassandra file classes 
def file_name(file_dir):
    for root, dirs, files in os.walk(file_dir):
         for file in files:
            if file[-4:] == '.txt':
                fileList.append(os.path.join(root, file))

In [5]:
file_name(file_dir)
fileList

['/content/drive/MyDrive/delesmell/dataset_class/cassandra-trunk/EndpointsByReplica.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/cassandra-trunk/Ec2MultiRegionSnitch.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/cassandra-trunk/BootStrapper.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/cassandra-trunk/ImmediateExecutor.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/cassandra-trunk/InvalidRequestException.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/cassandra-trunk/SnapshotDetailsTabularData.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/cassandra-trunk/CompactionStrategyHolder.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/cassandra-trunk/CollectionKeyIndex.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/cassandra-trunk/AbstractConditions.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/cassandra-trunk/BufferDecoratedKey.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/cassandra-trunk/AggregateFunctionSele

In [6]:
class preprocessing():
  

  def lm_find_unchinese(self ,line):
    pattern = re.compile(r'[\u4e00-\u9fa5]')
    unchinese = re.sub(pattern,"",line) #Exclude Chinese characters
    unchinese = re.sub('[{}]'.format(punctuation),"",unchinese) #Exclude Chinese characters

    return unchinese
    
  def del_stopwords(self,line):
    dicts = {'\n': '','!': '', '"': '','：':'', '#': '', '$': '', '%': '', '&': '', "'": '', '(': '', ')': '', '*': '', '+': '', ',': '', '-': '', '.': '', '/': '', ':': '', ';': '', '<': '', '=': '', '>': '', '?': '', '@': '', '[': '', '\\': '', ']': '', '^': '', '_': '', '`': '', '{': '', '|': '', '}': '', '~': ''}
    punc_table = str.maketrans(dicts)
    new_line = line.translate(punc_table)

    return new_line

In [7]:
all_files_words = []
class_name = []
Brain_Class = []
for i, f in enumerate(fileList):
    with open(f,encoding='gb18030',errors='ignore') as file:
        filewords = []
        list2 = []
        is_brain = 0
        filename = os.path.basename(file.name) #example: Ground.txt
        name = filename[:-4] #example: Ground
        class_name.append(name)
        filewords.append(name) # file names
        list2.append(name)
        pre = preprocessing()
        for line in file.readlines():
            line = pre.lm_find_unchinese(line)
            line = pre.del_stopwords(line)
            values = line.split(' ')
            coefs = values[1:-1]
            coefs = [i for i in coefs if i != '']
            keyword_brain = 'hongshuai'            

            if  keyword_brain in line:
                is_brain = 1
            if len(coefs) != 0:
                if coefs != ' ':
                    if coefs != '\n':
                        for i in coefs:
                            filewords.append(i)

        all_files_words.append(filewords)
        
        if is_brain == 0:
            list2.append(False)
        else :
            list2.append(True)
        
        Brain_Class.append(list2)

In [9]:
Brain_Class = np.array(Brain_Class)
news_df = np.array(all_files_words,dtype=object) 
print(news_df)#a list that contains files words in list (a list of lists) 

[list(['EndpointsByReplica', 'EndpointsByReplicaMapReplicaEndpointsForRange', 'EndpointsForRange', 'getReplica', 'return'])
 list(['Ec2MultiRegionSnitch', 'Ec2MultiRegionSnitch', 'throws', 'IOException', 'ConfigurationException', 'InetAddress', 'loggerinfoEC2Snitch', 'using', 'publicIP', 'as', 'identifier', 'if', 'DatabaseDescriptorgetBroadcastRpcAddress', 'null', 'loggerinfobroadcastrpcaddress', 'unset', 'broadcasting', 'public', 'IP', 'as', 'rpcaddress'])
 list(['BootStrapper', 'CollectionToken', 'allocateTokensfinal', 'TokenMetadata', 'metadataInetAddressAndPort', 'addressint', 'rfint', 'numTokenslong', 'if', 'FBUtilitiesgetBroadcastAddressAndPortequalsInetAddressAndPortgetLoopbackAddress', 'CollectionToken', 'return', 'CollectionToken', 'allocateTokensfinal', 'TokenMetadata', 'metadataInetAddressAndPort', 'addressString', 'allocationKeyspaceint', 'numTokenslong', 'if', 'FBUtilitiesgetBroadcastAddressAndPortequalsInetAddressAndPortgetLoopbackAddress', 'Keyspace', 'if', 'ks', 'null',

In [16]:
from keras import Input,Model
from keras.layers import Dense
class SemanticAnalysis():

  def calculate_acsm( file_words, n):
    vectorizer = CountVectorizer() #used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text.
    X = vectorizer.fit_transform(file_words) #for every word in file_words this function index then and give each word a uniq id 


    n_cap = 1
    svd_model = TruncatedSVD(n_components=n_cap)
    X2 = svd_model.fit_transform(X) # convert each word in to a number 

    D = svd_model.components_ #The right singular vectors of the input data.
    T = X2
    S = np.diagflat(svd_model.singular_values_) #one number

    X = np.dot(T, S)
    X = np.dot(X, D)
    c = np.dot(X, X.T)

    acsm = None
    sum = 0
    for i in c:
        for j in i:
            if j < 0:
                j = 0
            sum += j
        acsm = sum / n / n


    return acsm

  def additem(list):
      if len(list) < 3:
          q = len(list) - 1
          str = '_1'
          str2 = '_2'
          item = ''.join([list[q], str])
          item_2 = ''.join([list[q], str2])
          list.append(item)
          list.append(item_2)

      return list
  def word2vec_train(class_words):
        classtovec = []
        classtovec.append(class_words[0])
        uniq_words = []

        additem(class_words)

        #get uniq words
        for word in class_words:
            if word not in uniq_words:
                uniq_words.append(word)

        #Then we assign a number to each word and create word2int dictionary 
        word2int = {}
        for i,word in enumerate(uniq_words):
            word2int[word] = i




        WINDOW_SIZE=2
        data = []
        # for sentence in class_words:
        for idx,word in enumerate(class_words):
            for neighbor in class_words[max(idx - WINDOW_SIZE,0) : min(idx + WINDOW_SIZE,len(class_words))+ 1]:
                if neighbor != word:
                    data.append([word, neighbor])
        #We create a data frame that contains a word and its 2 following neighbors.
        df = pd.DataFrame(data,columns=['input','label'])



        ONE_HOT_DIM = len(uniq_words)
        #function to convert numbers to one hot vectors
        def to_one_hot_encoding(data_point_index):
            one_hot_encoding = np.zeros(ONE_HOT_DIM)
            one_hot_encoding[data_point_index] = 1
            return one_hot_encoding

        X = []
        Y = []

        for x, y in zip(df['input'],df['label']):
            X.append(to_one_hot_encoding(word2int[x]))
            Y.append(to_one_hot_encoding(word2int[y]))

        x_train = np.array(X)
        y_train = np.array(Y)


        #Defining the size of the embedding
        embed_size = 3
        xx = Input(shape=(x_train.shape[1],))
        yy = Dense(units=embed_size, activation='linear')(xx)
        yy =Dense(units=y_train.shape[1],activation='softmax')(yy)
        model = Model(inputs=xx, outputs=yy)
        model.compile(loss = 'categorical_crossentropy',optimizer = 'adam')
        #optimizing the network weights
        model.fit(x=x_train,y=y_train,
            batch_size=128,
            epochs=500
        )

        weights = model.get_weights()[0]



        for word in uniq_words:
            q = weights[df.get(word)]
            arr_new = np.sum(q, axis=1)
            arr_new = arr_new.tolist()

            classtovec.append(arr_new)

            break

        return classtovec

In [11]:
Brain_Class

array([['EndpointsByReplica', 'False'],
       ['Ec2MultiRegionSnitch', 'False'],
       ['BootStrapper', 'False'],
       ...,
       ['CreateKeyspaceStatement', 'False'],
       ['DropTypeStatement', 'False'],
       ['AlterRoleStatement', 'False']], dtype='<U41')

In [12]:
for k in range(0,len(Brain_Class)):
    a = Brain_Class[k][0]
    if '_' in a:
        n = a.find('_')
        new_str = ''
        for i in range(0, n):
            new_str = new_str + a[i]
        Brain_Class[k][0] = new_str
    print(Brain_Class[k][0])

EndpointsByReplica
Ec2MultiRegionSnitch
BootStrapper
ImmediateExecutor
InvalidRequestException
SnapshotDetailsTabularData
CompactionStrategyHolder
CollectionKeyIndex
AbstractConditions
BufferDecoratedKey
AggregateFunctionSelector
CassandraValidationIterator
RingPosition
ArrayClusteringBoundary
CreateFunctionStatement
CassandraException
LastEventIdBroadcasterMBean
DropKeyspaceStatement
MorePartitions
JMXEnabledThreadPoolExecutorMBean
AuthenticationStatement
AuthKeyspace
DoubleType
DropTableStatement
DateType
FSError
ArrayClusteringBoundOrBoundary
Batch
TableAttributes
CorruptSSTableException
IndexNotAvailableException
GossiperEvent
CqlOutputFormat
IndexSummary
SchemaElement
CreateViewStatement
HintsServiceDiagnostics
AbstractCompactionController
CASClientWriteRequestMetrics
CachesTable
AbstractGettableByIndexData
AbstractArrayClusteringPrefix
RangeIntersectionIterator
KeyspaceAttributes
VersionGenerator
HintEvent
CassandraCompressedStreamWriter
AlibabaCloudSnitch
BufferClusteringBound
I

In [ ]:
len(index)

1047

In [26]:
def additem(list):
    if len(list)<3:
        q = len(list)-1
        str = '_1'
        str2 = '_2'
        item = ''.join([list[q], str])
        item_2 = ''.join([list[q], str2])
        # item = list[q]
        list.append(item)
        list.append(item_2)
        # print(list)
    return list


In [17]:
q = 0
acsm= []
for file_words in all_files_words:
    lists = []
    for word in file_words:
        lists.append(word)

    if len(lists) > 1:
        acsm.append(SemanticAnalysis.calculate_acsm(lists,len(file_words)))
    else :
        acsm.append(1)

In [18]:
class_name

['EndpointsByReplica',
 'Ec2MultiRegionSnitch',
 'BootStrapper',
 'ImmediateExecutor',
 'InvalidRequestException',
 'SnapshotDetailsTabularData',
 'CompactionStrategyHolder',
 'CollectionKeyIndex',
 'AbstractConditions',
 'BufferDecoratedKey',
 'AggregateFunctionSelector',
 'CassandraValidationIterator',
 'RingPosition',
 'ArrayClusteringBoundary',
 'CreateFunctionStatement',
 'CassandraException',
 'LastEventIdBroadcasterMBean',
 'DropKeyspaceStatement',
 'MorePartitions',
 'JMXEnabledThreadPoolExecutorMBean',
 'AuthenticationStatement',
 'AuthKeyspace',
 'DoubleType',
 'DropTableStatement',
 'DateType',
 'FSError',
 'ArrayClusteringBoundOrBoundary',
 'Batch',
 'TableAttributes',
 'CorruptSSTableException',
 'IndexNotAvailableException',
 'GossiperEvent',
 'CqlOutputFormat',
 'IndexSummary',
 'SchemaElement',
 'CreateViewStatement',
 'HintsServiceDiagnostics',
 'AbstractCompactionController',
 'CASClientWriteRequestMetrics',
 'CachesTable',
 'AbstractGettableByIndexData',
 'AbstractAr

In [19]:
classname_acsm = []
for i in range(len(class_name)):
            t=[class_name[i],acsm[i]]
            classname_acsm.append(t)

classname_acsm = np.array(classname_acsm)


In [20]:
classname_acsm

array([['EndpointsByReplica', '0.08844603979453364'],
       ['Ec2MultiRegionSnitch', '0.01850025291424763'],
       ['BootStrapper', '0.03572804983931681'],
       ...,
       ['CreateKeyspaceStatement', '0.0664345194444439'],
       ['DropTypeStatement', '0.06250000000000003'],
       ['AlterRoleStatement', '0.22222222222222243']], dtype='<U41')

In [21]:
df =pd.read_csv("/content/drive/MyDrive/delesmell/dataset_class/cassandra-trunk/cassandra.csv") 

In [22]:
df_name= df.iloc[:, 0]

In [23]:
#add new column
ngList=[] #Prepare a list and store the data of the new column in it

for name in df_name:

    q = 1
    for i in range(0,len(classname_acsm)):
        if name == classname_acsm[i][0]:
            ngList.append(classname_acsm[i][1])
            q = 0

    if q==1 :
        ngList.append(1)

In [24]:
brainList=[] #Prepare a list and store the data of the new column in it

for name in df_name:
#     print(index)
    q = 1
    for i in range(0,len(classname_acsm)):
        if name == (Brain_Class[i][0]):
            brainList.append(Brain_Class[i][1])
            q = 0
    if q==1 :
        brainList.append('0')
        

In [27]:
embedding = []
for class_words in all_files_words:
  embedding.append(SemanticAnalysis.word2vec_train(class_words))
  break
print(embedding)

Epoch 1/500
1/1 [==============================] - 1s 573ms/step - loss: 1.6536
Epoch 2/500
1/1 [==============================] - 0s 11ms/step - loss: 1.6522
Epoch 3/500
1/1 [==============================] - 0s 10ms/step - loss: 1.6507
Epoch 4/500
1/1 [==============================] - 0s 10ms/step - loss: 1.6492
Epoch 5/500
1/1 [==============================] - 0s 7ms/step - loss: 1.6477
Epoch 6/500
1/1 [==============================] - 0s 11ms/step - loss: 1.6463
Epoch 7/500
1/1 [==============================] - 0s 7ms/step - loss: 1.6448
Epoch 8/500
1/1 [==============================] - 0s 13ms/step - loss: 1.6434
Epoch 9/500
1/1 [==============================] - 0s 12ms/step - loss: 1.6419
Epoch 10/500
1/1 [==============================] - 0s 6ms/step - loss: 1.6405
Epoch 11/500
1/1 [==============================] - 0s 6ms/step - loss: 1.6391
Epoch 12/500
1/1 [==============================] - 0s 6ms/step - loss: 1.6376
Epoch 13/500
1/1 [==============================] - 0

In [28]:
emblist=[]
for index in df_name:
#     print(index)
    q = 1
    for i in range(0,len(embedding)):
        if index == embedding[i][0]:
            # embedding[i][1].reshape((1,3))
            # l = embedding[i][1].tolist()
            emblist.append(embedding[i][1])
            q = 0
            # break
#             print('*')
    if q==1 :
        emblist.append([[1, 1, 1]])

In [29]:
word2vec = []
list1 = []
list2 = []
list3 = []
for i in range(0,len(emblist)):
    word2vec.append(emblist[i][0])

for i in range(0,len(word2vec)):
    list1.append(word2vec[i][0])
for i in range(0,len(word2vec)):
    list2.append(word2vec[i][1])
for i in range(0,len(word2vec)):
    list3.append(word2vec[i][2])

In [30]:
column=df.columns.tolist()

In [31]:
df['acsm'] = ngList 
df['is_brain'] = brainList 
df.to_csv(save_file_csv,mode = 'a',index =False)

In [32]:
df.head()

,NAME,AMW,ATDF,BOVM,BUR,CBO,CC,CM,CRIX,DAC,...,NOA,NOAM,NOD,NOM,NOPA,TCC,WMC,WOC,acsm,is_brain
0,AuditTrigger,1.00,0,0,1.0,2,0,0,0.0,0,...,1,0,2,0,0.00,2,1.0,NaN,1,0
1,ITrigger,0.00,0,0,1.0,0,0,0,0.0,0,...,0,5,1,0,1.00,0,1.0,NaN,1,0
2,CustomClassLoader,1.40,0,0,1.0,1,0,0,0.0,0,...,3,0,5,0,0.67,7,1.0,NaN,1,0
3,1,1.00,0,0,1.0,0,0,0,0.0,0,...,0,0,1,0,1.00,1,1.0,NaN,1,0
4,TriggersTest,1.08,0,0,1.0,2,0,0,0.0,0,...,4,0,13,0,0.71,14,1.0,NaN,1,0


In [33]:
df['word2vec'] = list1
df['word2vec2'] = list2
df['word2vec3'] = list3

In [34]:
df.head()

,NAME,AMW,ATDF,BOVM,BUR,CBO,CC,CM,CRIX,DAC,...,NOM,NOPA,TCC,WMC,WOC,acsm,is_brain,word2vec,word2vec2,word2vec3
0,AuditTrigger,1.00,0,0,1.0,2,0,0,0.0,0,...,0,0.00,2,1.0,NaN,1,0,1,1,1
1,ITrigger,0.00,0,0,1.0,0,0,0,0.0,0,...,0,1.00,0,1.0,NaN,1,0,1,1,1
2,CustomClassLoader,1.40,0,0,1.0,1,0,0,0.0,0,...,0,0.67,7,1.0,NaN,1,0,1,1,1
3,1,1.00,0,0,1.0,0,0,0,0.0,0,...,0,1.00,1,1.0,NaN,1,0,1,1,1
4,TriggersTest,1.08,0,0,1.0,2,0,0,0.0,0,...,0,0.71,14,1.0,NaN,1,0,1,1,1
